# Proyecto

Objetivo: Analizar la seguridad de contraseñas con base en la lista de contraseñas y su frecuencia de uso

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from numpy import random
from torch.multiprocessing import Process, Queue, Lock
import multiprocessing as mp
import random


In [2]:
data =pd.read_csv('passwords/pass (1).csv')
for i in range(2,100):
    filename = pd.read_csv('passwords/pass ('+str(i)+').csv')
    data = pd.concat([data,filename],axis=0)
data

,password,hash,count
0,1980290,083040BB6B1D95F2B8AC447B50D21DAB38DAFF16,13
1,tornadof,0A7709AD9837766F20156FF016ABF66F8036ABC1,13
2,vova87654,66221A7691CDC27D9D31071A0CFA0712C100C7D7,13
3,XpKvShrO,4CB7086C94AE4D82E519FF8F98D0DA1DA38DBE53,13
4,tvjgtl,9E1FD26A17D2EBE7C9FD4181EF49424C7091D484,13
...,...,...,...
6840,491144,4511C39CDA1EFE76964C79783B52053BA7CD09DD,69
6841,xeyfhm3477362341,E2EC663D721E9F22F5CF39D5AA27A89CE00172D0,69
6842,irontree4,350AA827594B47AF3BCB9256E2388EFB958C4529,69
6843,wh2t3v3r,221C76F1C2AD5883003AF5EB7A495A41580B87C5,69


In [3]:
data.isnull().sum()

password    2
hash        0
count       0
dtype: int64

In [4]:
data = data.dropna(axis=0)
data.isnull().sum()

password    0
hash        0
count       0
dtype: int64

In [5]:
data = data.groupby(data['password']).agg({'count':sum}).reset_index()
print(data.shape)
data.head()

(677566, 2)


,password,count
0,!!!!!,401
1,!!!!!!,2304
2,!!!!!!!,386
3,!12345,312
4,!23456,347


In [6]:
data = data.sort_values(by='count',ascending=False)
data.head()

,password,count
43879,123456789,7016669
506981,qwerty,3599486
35227,111111,2900049
43856,12345678,2680521
223879,abc123,2670319


Ahora vamos a establecer una métrica para pasar de la cuenta de las contraseñas a una métrica de su seguridad, para esto se decide hacer un escalamiento estándar al inverso de la cuenta de contraseñas. Esto para darle una mayor scoring a aquellos que tienen menos apariciones.

In [7]:
pass_count = data.iloc[:,1].values
pass_count

array([7016669, 3599486, 2900049, ...,      12,      12,      12])

In [8]:
from copy import copy
passes = copy(data.iloc[:,0].values.tolist())
passes[:5]

['123456789', 'qwerty', '111111', '12345678', 'abc123']

In [9]:
data = None  #Liberamos memoria

Dividimos el conjunto de la seguridad de las contraseñas uniformemente en `n` intervalos

In [10]:
n = 10


intervalo = pass_count.shape[0]//n

pass_security = np.zeros(len(passes))
intervalo_idx = 0
for i in range(n):
    for j in range(intervalo_idx,intervalo_idx+intervalo):
        pass_security[j] = i
        intervalo_idx += 1

for j in range(intervalo_idx,len(pass_count)):
    pass_security[j] = n
    
pass_security = pass_security.astype(int)   
pass_security

array([ 0,  0,  0, ..., 10, 10, 10])

A continuación, hacemos una identificación de cada carácter que aparece en las contraseñas, esto para después poder identificar cada contraseña como una codificación de los carácteres que la componen

In [11]:
dict_letras = {}

idx = 0
for i in range(len(passes)):
    #passes[i] = list(passes[i])
    for letter in passes[i]:
        if letter not in dict_letras.keys():
            dict_letras[letter] = idx
            idx += 1

In [12]:
dict_letras.items()

dict_items([('1', 0), ('2', 1), ('3', 2), ('4', 3), ('5', 4), ('6', 5), ('7', 6), ('8', 7), ('9', 8), ('q', 9), ('w', 10), ('e', 11), ('r', 12), ('t', 13), ('y', 14), ('a', 15), ('b', 16), ('c', 17), ('0', 18), ('i', 19), ('l', 20), ('o', 21), ('v', 22), ('u', 23), ('p', 24), ('m', 25), ('n', 26), ('k', 27), ('d', 28), ('g', 29), ('z', 30), ('s', 31), ('x', 32), ('f', 33), ('h', 34), ('j', 35), ('S', 36), ('V', 37), ('Q', 38), ('B', 39), ('L', 40), ('P', 41), ('J', 42), ('E', 43), ('!', 44), ('A', 45), ('W', 46), ('O', 47), ('R', 48), ('D', 49), ('G', 50), ('.', 51), ('@', 52), ('Y', 53), ('I', 54), ('?', 55), ('T', 56), ('U', 57), ('*', 58), ('M', 59), ('C', 60), ('-', 61), ('F', 62), ('H', 63), ('Z', 64), ('$', 65), ('#', 66), ('X', 67), ('N', 68), ('K', 69), (';', 70), ('%', 71), ('^', 72), ('_', 73), ('&', 74), ("'", 75), ('+', 76), ('~', 77), ('|', 78), ('{', 79), ('}', 80), ('`', 81)])

A continuación definimos algunas funciones auxiliares para las neuronas de **entrada**:

In [13]:
dict_size = len(dict_letras.values())

def password_to_encode(password):
    '''
    Función que dada una contraseña, devuelve el encoding acomodado al tratamiento
    que tuvieron los datos de este documento.
    --------------------------------------------------------------
    :param password str: Cadena de texto que contiene la contraseña a convertir
    
    :returns array: Vector 27-dimensional codificado. 
    '''
    pass_vector = list(map(ord,list(password)))
    pass_vector += [min_string]*(max_len_pass-len(pass_vector))
    pass_vector = np.array(pass_vector)
    #pass_vector = ms.transform(pass_vector.reshape(-1,27))
    
    return pass_vector

#prueba = password_to_encode().reshape(-1,27)
#neigh2.predict(prueba)

In [14]:
max_len_pass = 0 # Vamos guardando la longitud de contraseña más grande
min_string = 1000 # Guardamos el ASCII más pequeño que nos encontramos

for i in range(len(passes)):
    passes[i] = list(map(ord,list(passes[i])))
    if len(passes[i]) > max_len_pass:
        max_len_pass = len(passes[i])
    
    for letter_ascii in passes[i]:
        if letter_ascii < min_string:
            min_string = letter_ascii

min_string -= 100 # min_string será el valor para el espacio vacío

for i in range(len(passes)):
    n = len(passes[i])
    #Llenamos cada contraseña para que tengan la misma longitud
    passes[i] = passes[i] + [min_string]*(max_len_pass-len(passes[i]))
    
    
passes = np.array(passes)
passes.shape

(677566, 27)

In [15]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(np.array(passes), pass_security, test_size=0.2)

In [16]:
X_train

array([[ 51,  48,  48, ..., -67, -67, -67],
       [114, 101,  97, ..., -67, -67, -67],
       [108, 105, 112, ..., -67, -67, -67],
       ...,
       [121, 111, 117, ..., -67, -67, -67],
       [ 50,  53,  56, ..., -67, -67, -67],
       [100, 119, 105, ..., -67, -67, -67]])

In [17]:
from sklearn.neighbors import KNeighborsClassifier

neigh = KNeighborsClassifier(n_neighbors=5)


Primer modelo: 

In [19]:
neigh.fit(X_train,y_train)

KNeighborsClassifier()

In [22]:
neigh.score(X_test,y_test)

0.23392417019643727

Segundo modelo, 50 vecinos, métrica de hamming:

In [23]:
neigh2 = KNeighborsClassifier(n_neighbors=50,metric='hamming',algorithm='ball_tree',
                             n_jobs=-1)
neigh2.fit(X_train,y_train)
neigh2.score(X_test,y_test)

0.210214442788199

In [27]:
suma = 0
for i in range(X_test.shape[0]):
    prediccion = neigh2.predict(X_test[i].reshape(1,-1))[0]
    suma += abs(y_test[i]-prediccion)
suma/X_test.shape[0]

2.613456912200953

In [30]:
print('La desviación estándar de la diferencia de las clases predichas con \n respecto las clases verdadereas es:',suma/X_test.shape[0])

La desviación estándar de la diferencia de las clases predichas con 
 respecto las clases verdadereas es: 2.613456912200953


Tercer modelo, 50 vecinos, métrica euclidea:

In [21]:
neigh3 = KNeighborsClassifier(n_neighbors=200)
neigh3.fit(X_train,y_train)
neigh3.score(X_test,y_test)

0.22700237613825877

In [22]:
suma3 = 0
for i in range(X_test.shape[0]):
    prediccion = neigh3.predict(X_test[i].reshape(1,-1))[0]
    suma3 += abs(y_test[i]-prediccion)
#suma3/X_test.shape[0]

In [23]:
print('La desviación estándar de la diferencia de las clases predichas con \n respecto las clases verdadereas es:',suma3/X_test.shape[0])

La desviación estándar de la diferencia de las clases predichas con 
 respecto las clases verdadereas es: 2.3098941806750592


Con la información obtenida, resulta que la mejor aproximación a los datos verdaderos se da por el último modelo, teniendo en promedio, un error de 2.25 puntos de diferencia entre los datos predichos y la clase verdadera. A continuación procederemos a proponer un modelo para generar contraseñas seguras y las verificaremos con el último modelo.

Este modelo plantea el uso de generación de altos valores de entropía entre palabras comunes para una mejor seguridad. Más información sobre este modelo pueden ser encontrados en https://theworld.com/~reinhold/diceware.html

In [24]:
def get_pass(n, print_valor = False):
    data = pd.read_csv("Diceware/my_dicewareEsp.csv")
    palabras = data.sample(n)
    if print_valor : print(palabras)
    passphrase = ""
    for val in palabras["valor"]:
        passphrase = passphrase + val +"_"
    return passphrase[:-1]

In [25]:
for _ in range(5):
    intentos = 0
    while True:
        intentos += 1
        contrasena = get_pass(3).replace("_","")
        code = password_to_encode(contrasena).reshape(1,-1)
        security = neigh3.predict(code)
        
        if security > 6:
            print('Contraseña sugerida:',contrasena,'con seguridad',security,'+/-2\n      En',intentos,'intento(s)')
            break

Contraseña sugerida: moyak78mitra con seguridad [9] +/-2
      En 1 intento(s)
Contraseña sugerida: ibish15624 con seguridad [8] +/-2
      En 1 intento(s)
Contraseña sugerida: lunarlombav92 con seguridad [9] +/-2
      En 1 intento(s)
Contraseña sugerida: fugazlusoruedo con seguridad [8] +/-2
      En 1 intento(s)
Contraseña sugerida: 460temerretor con seguridad [9] +/-2
      En 1 intento(s)


In [26]:
intentos_totales = 0
for _ in range(500):
    intentos = 0
    while True:
        intentos += 1
        try:
            contrasena = get_pass(4).replace("_","")
        except Exception as e:
            continue
        code = password_to_encode(contrasena).reshape(1,-1)
        security = neigh3.predict(code)
        
        if security > 8:
            intentos_totales += intentos
            break
intentos_totales /= 500
print('En promedio, se tardó: ',intentos_totales,' en generar contraseñas seguras')

En promedio, se tardó:  1.022  en generar contraseñas seguras
